# Task 15.5 Solution: Reproducibility Audit

This notebook provides solutions for the reproducibility verification exercise.

---

## Exercise Solution: Verify Training Reproducibility

**Task:** Verify that training is reproducible and observe what happens when deterministic settings are disabled.

Requirements:
1. Run `reproducible_training_run` with seed=123, 3 times
2. Verify all three runs have identical final metrics
3. Disable deterministic mode and compare
4. Document findings

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from typing import Dict, Any

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
def set_seed(seed: int, deterministic: bool = True) -> None:
    """
    Set all random seeds for reproducibility.
    
    Args:
        seed: The random seed to use
        deterministic: If True, use deterministic algorithms
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        try:
            torch.use_deterministic_algorithms(True)
        except AttributeError:
            pass
    else:
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True
        try:
            torch.use_deterministic_algorithms(False)
        except AttributeError:
            pass

In [ ]:
class SimpleNet(nn.Module):
    """Simple network for reproducibility testing."""
    def __init__(self, input_size=10, hidden_size=20, output_size=2):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

def create_data(seed: int, n_samples: int = 1000):
    """Create reproducible synthetic data."""
    set_seed(seed)
    X = torch.randn(n_samples, 10)
    y = (X[:, 0] + X[:, 1] > 0).long()
    return X, y

In [ ]:
def train_model(
    seed: int,
    deterministic: bool = True,
    n_epochs: int = 5,
    learning_rate: float = 0.01,
    batch_size: int = 32
) -> Dict[str, Any]:
    """
    Train a model with configurable reproducibility settings.
    
    Args:
        seed: Random seed
        deterministic: Whether to use deterministic algorithms
        n_epochs: Number of training epochs
        learning_rate: Learning rate
        batch_size: Batch size
    
    Returns:
        Dictionary with training results
    """
    # Set seed with deterministic settings
    set_seed(seed, deterministic=deterministic)
    
    # Create data
    X, y = create_data(seed)
    
    # Create DataLoader with seeded generator
    g = torch.Generator()
    g.manual_seed(seed)
    
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        generator=g
    )
    
    # Create model (seeded initialization)
    model = SimpleNet()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    history = []
    
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)
        
        avg_loss = total_loss / len(dataloader)
        accuracy = correct / total
        history.append({"epoch": epoch, "loss": avg_loss, "accuracy": accuracy})
    
    # Get final weights hash for comparison
    weights_hash = hash(tuple(model.fc1.weight.flatten().tolist()[:10]))
    
    return {
        "final_loss": history[-1]["loss"],
        "final_accuracy": history[-1]["accuracy"],
        "history": history,
        "weights_hash": weights_hash,
        "deterministic": deterministic
    }

In [ ]:
# Step 1: Run 3 times with seed=123 (deterministic mode)

SEED = 123
N_RUNS = 3

print("="*60)
print("EXPERIMENT 1: DETERMINISTIC MODE (Expected: Identical Results)")
print("="*60)

deterministic_results = []

for i in range(N_RUNS):
    result = train_model(seed=SEED, deterministic=True)
    deterministic_results.append(result)
    print(f"\nRun {i+1}:")
    print(f"   Final Loss: {result['final_loss']:.6f}")
    print(f"   Final Accuracy: {result['final_accuracy']:.6f}")
    print(f"   Weights Hash: {result['weights_hash']}")

In [ ]:
# Step 2: Verify all three runs have identical results

print("\n" + "="*60)
print("VERIFICATION: Are Results Identical?")
print("="*60)

losses = [r['final_loss'] for r in deterministic_results]
accuracies = [r['final_accuracy'] for r in deterministic_results]
hashes = [r['weights_hash'] for r in deterministic_results]

loss_identical = all(abs(l - losses[0]) < 1e-9 for l in losses)
acc_identical = all(abs(a - accuracies[0]) < 1e-9 for a in accuracies)
hash_identical = len(set(hashes)) == 1

print(f"\n{'Metric':<20} {'Status':<10} {'Values'}")
print("-" * 60)
print(f"{'Final Loss':<20} {'✅ SAME' if loss_identical else '❌ DIFF':<10} {losses[0]:.6f}")
print(f"{'Final Accuracy':<20} {'✅ SAME' if acc_identical else '❌ DIFF':<10} {accuracies[0]:.6f}")
print(f"{'Weights Hash':<20} {'✅ SAME' if hash_identical else '❌ DIFF':<10} {hashes[0]}")

all_identical = loss_identical and acc_identical and hash_identical
print(f"\n🎯 Overall: {'✅ FULLY REPRODUCIBLE' if all_identical else '❌ NOT REPRODUCIBLE'}")

In [ ]:
# Step 3: Disable deterministic mode and compare

print("\n" + "="*60)
print("EXPERIMENT 2: NON-DETERMINISTIC MODE (Expected: Varying Results)")
print("="*60)

non_deterministic_results = []

for i in range(N_RUNS):
    result = train_model(seed=SEED, deterministic=False)
    non_deterministic_results.append(result)
    print(f"\nRun {i+1}:")
    print(f"   Final Loss: {result['final_loss']:.6f}")
    print(f"   Final Accuracy: {result['final_accuracy']:.6f}")
    print(f"   Weights Hash: {result['weights_hash']}")

In [ ]:
# Compare non-deterministic results

print("\n" + "="*60)
print("COMPARISON: Non-Deterministic Results")
print("="*60)

nd_losses = [r['final_loss'] for r in non_deterministic_results]
nd_accuracies = [r['final_accuracy'] for r in non_deterministic_results]
nd_hashes = [r['weights_hash'] for r in non_deterministic_results]

nd_loss_identical = all(abs(l - nd_losses[0]) < 1e-9 for l in nd_losses)
nd_acc_identical = all(abs(a - nd_accuracies[0]) < 1e-9 for a in nd_accuracies)
nd_hash_identical = len(set(nd_hashes)) == 1

print(f"\n{'Metric':<20} {'Status':<10} {'Min':<12} {'Max':<12} {'Range'}")
print("-" * 70)
print(f"{'Final Loss':<20} {'✅ SAME' if nd_loss_identical else '⚠️ VARIES':<10} "
      f"{min(nd_losses):.6f}   {max(nd_losses):.6f}   {max(nd_losses)-min(nd_losses):.6f}")
print(f"{'Final Accuracy':<20} {'✅ SAME' if nd_acc_identical else '⚠️ VARIES':<10} "
      f"{min(nd_accuracies):.6f}   {max(nd_accuracies):.6f}   {max(nd_accuracies)-min(nd_accuracies):.6f}")
print(f"{'Weights Hash':<20} {'✅ SAME' if nd_hash_identical else '⚠️ VARIES':<10} "
      f"{len(set(nd_hashes))} unique values")

In [ ]:
# Step 4: Document findings

print("\n" + "="*60)
print("FINDINGS & DOCUMENTATION")
print("="*60)

findings = f"""
📋 REPRODUCIBILITY EXPERIMENT REPORT

Seed Used: {SEED}
Number of Runs: {N_RUNS}
Model: SimpleNet (10->20->2)
Training: 5 epochs, lr=0.01, batch_size=32

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

EXPERIMENT 1: DETERMINISTIC MODE
  Settings:
    - torch.backends.cudnn.deterministic = True
    - torch.backends.cudnn.benchmark = False
    - torch.use_deterministic_algorithms(True)
  
  Results:
    - Final Loss: {deterministic_results[0]['final_loss']:.6f} (identical across runs: {'YES' if loss_identical else 'NO'})
    - Final Accuracy: {deterministic_results[0]['final_accuracy']:.6f} (identical: {'YES' if acc_identical else 'NO'})
    - Weights: {'IDENTICAL' if hash_identical else 'VARYING'}
  
  Conclusion: {'✅ FULLY REPRODUCIBLE' if all_identical else '❌ NOT REPRODUCIBLE'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

EXPERIMENT 2: NON-DETERMINISTIC MODE
  Settings:
    - torch.backends.cudnn.deterministic = False
    - torch.backends.cudnn.benchmark = True
    - Deterministic algorithms disabled
  
  Results:
    - Loss Range: {min(nd_losses):.6f} to {max(nd_losses):.6f} (Δ={max(nd_losses)-min(nd_losses):.6f})
    - Accuracy Range: {min(nd_accuracies):.6f} to {max(nd_accuracies):.6f} (Δ={max(nd_accuracies)-min(nd_accuracies):.6f})
    - Unique Weight Hashes: {len(set(nd_hashes))}
  
  Conclusion: ⚠️ RESULTS VARY BETWEEN RUNS

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

KEY INSIGHTS:

1. DETERMINISTIC MODE IS ESSENTIAL for reproducibility
   - Same seed + deterministic settings = identical results
   - This is crucial for research papers and debugging

2. NON-DETERMINISTIC MODE trades reproducibility for speed
   - CUDNN benchmark mode finds faster algorithms
   - Results vary due to non-deterministic GPU operations

3. VARIATION MAGNITUDE
   - Accuracy varied by ~{max(nd_accuracies)-min(nd_accuracies):.2%}
   - This seems small but matters for reproducibility

4. RECOMMENDATIONS:
   - Use deterministic mode for: research, debugging, CI/CD tests
   - Use non-deterministic for: production training (faster)
   - Always document which mode was used
"""

print(findings)

In [ ]:
# Visualize the comparison
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Loss comparison
runs = range(1, N_RUNS + 1)
ax1 = axes[0]
ax1.bar([r - 0.2 for r in runs], losses, width=0.35, label='Deterministic', color='green', alpha=0.8)
ax1.bar([r + 0.2 for r in runs], nd_losses, width=0.35, label='Non-Deterministic', color='orange', alpha=0.8)
ax1.set_xlabel('Run Number')
ax1.set_ylabel('Final Loss')
ax1.set_title('Loss Comparison Across Runs')
ax1.legend()
ax1.set_xticks(runs)

# Plot 2: Accuracy comparison
ax2 = axes[1]
ax2.bar([r - 0.2 for r in runs], [a * 100 for a in accuracies], width=0.35, 
        label='Deterministic', color='green', alpha=0.8)
ax2.bar([r + 0.2 for r in runs], [a * 100 for a in nd_accuracies], width=0.35, 
        label='Non-Deterministic', color='orange', alpha=0.8)
ax2.set_xlabel('Run Number')
ax2.set_ylabel('Final Accuracy (%)')
ax2.set_title('Accuracy Comparison Across Runs')
ax2.legend()
ax2.set_xticks(runs)

plt.tight_layout()
plt.savefig('/tmp/reproducibility_comparison.png', dpi=150)
plt.show()

print("\n📊 Visualization saved!")

---

## Summary

### What We Learned:

1. **Deterministic mode produces identical results** across runs with the same seed
2. **Non-deterministic mode causes variation** due to GPU algorithm selection
3. **The variation is small but measurable** - important for scientific reproducibility
4. **Setting multiple seeds is necessary** - Python, NumPy, PyTorch, CUDA all need to be seeded

### Best Practices:

1. Always set seeds at the start of experiments
2. Use deterministic mode for reproducible research
3. Document your random seeds and settings
4. Verify reproducibility by running experiments multiple times
5. Log seeds with MLflow or other experiment tracking

In [ ]:
print("\n✅ Solution complete!")